# LGBIO2070 — Laboratory session 3: Treatment planning in proton therapy

#### *Teaching staff: Eliot Peeters, Romain Schyns, Prof. John A. Lee*
#### Contacts :
   - eliot.peeters@uclouvain.be
   - romain.schyns@uclouvain.be

#### **DUE DATE: Sunday, May 25, at 23:59:59**

#### Authors: Student1, Student2

<a target="_blank" href="https://colab.research.google.com/github/MIRO-UCLouvain/LGBIO2070_lab3/blob/main/LGBIO2070%20-%20Lab%203.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
  </a>

## 1. Data import and treamtent

In this part of the lab, you will get familiar with the data used for the proton therapy plan optimization.

### 1.1 Data import with the DICOM format

Images used in the clinic are exchanged and registered in the DICOM format, a universal file format which registers patient data together with image data. The different data fields are accessible though multiple data "[tags](https://www.dicomlibrary.com/dicom/dicom-tags/)".

- **Q1.1** Use the python library [Pydicom](https://pydicom.github.io/pydicom/stable/) to extract the DICOM provided data

As you might notice, the DICOM data is not in Hounsfield Unit (HU). To get the HU back, you will need two DICOM tags: *slope* and *intercept*. You will also need to retrieve the DICOM resolution using the *pixel spacing* and *slice thickness* tags.

- **Q1.2** Use the DICOM tags to extract the necessary information

**Hint**: the website [Innolitics](https://dicom.innolitics.com/ciods) provide comprehensive and readable information on DICOM tags

- **Q1.3** Take a moment to examine the pixel spacing and slice thickness. What do these parameters represent, and where do they come from? Can you think of any issues that might occur when visualizing the CT scan in different planes? In which orientations might these problems be most noticeable?
> *Answer here*

In [ ]:
# Code here

Now that you have all the necessary information, you can convert the pixel values to HU and plot the CT scan.
- **Q2.1** Convert the pixel values to HU using the correct formula.
- **Q2.2** Plot the three different views of the CT scan: axial, coronal, and sagittal. Make sure to keep the same aspect ratio across all views so that the images are accurately represented and comparable.



- **Q2.3** Are the slides in the right order? if not, reorder the slides correctly using a specific dicom tag.

- **Q2.4** Navigate through the different slides. Do you see anything that captures your attention? If so, where is it located, what might be causing it, and how could this issue be addressed or corrected? (If you prefer to navigate the scan through a graphical interface, you can use the [OHIF viewer](https://viewer-dev.ohif.org/localbasic) and upload the data). \
**Hint**: Look at the region of the mandible. (You do not need to implement your correction strategy)

> *Answer here*




In [ ]:
# Code here

## 1.2 Data for treatment planning optimization

As using a 3D CT scan for optimization will be too computationally expensive and the conversion of the data to a 3D matrix will be too memory consuming, we will use a 2D slice of the CT scan for the optimization.

 The conversion of the DICOM data to usable numpy arrays can be quite cumbersome; therefore, your nice teaching assistants provided you with compressed numpy arrays concatenated into a dictionary for the segmented organs and numpy arrays for the CT and the beamlet matrix. To access the data, you will need to use the following code:

```python
import numpy as np
# The segmented organs
RTSTUCTS_phase_0 = np.load("data/RTSTRUCT_dict_phase_0.npz", allow_pickle=True)
RTSTUCTS_phase_5 = np.load("data/RTSTRUCT_dict_phase_5.npz", allow_pickle=True)

#example of the RTSTRUCTS dictionary
CTV = RTSTUCTS_phase_0["CTV"]

# The CT scans
ct_phase_0 = np.load("data/CT_phase_0.npy",allow_pickle=True)
ct_phase_5 = np.load("data/CT_phase_5.npy",allow_pickle=True)

# The beamlet matrixes
beamlets_phase_0 = np.load("data/Beamlets_phase_0.npy",allow_pickle=True)
beamlets_phase_5 = np.load("data/Beamlets_phase_5.npy",allow_pickle=True)

```

In the RTSTRUCTS dictionary, you will find the following arrays (the arrays are boolean 512x512 mask) :

- CTV
- Heart
- Lungs
- Liver
- Spinal canal
- Body

As you can see, we provide you 2 slices of 2 different CT scans. Those slices originate from a 4DCT. A 4DCT that records multiple CT scans over time. Here we provide you with 2 different phases of the 4DCT. The first phase is the 0% respiratory phase and the second is the 50% phase.\

We will first use the 0% phase.
- **Q3.1** Explore and plot different HU values of the CT scans for -1000 HU, 0 HU, 500 HU, 1000 HU.
- **Q3.2** To what tissue correspond these HU values?
> *Answer here*

- **Q3.3** Display the different binary masks and overlay them on the CT

**Hint**: use [plt.countour](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contour.html)

In [ ]:
# code here

Now we will explore the pre-computed beamlets. Those have been computed though the use of a Monte-Carlo simulator embeded in the open-source treatment planning system [OpenTPS](https://opentps.org/).

- **Q4.1** Display several beamlets on the 0% phase CT scan. (**e. g.** beamlet index 50, 100, 150 and 200).

**Hint**: use the colormap 'jet' to display the beamlets.

**Hint 2**: each beamlet is stored in a 1D array, you will need to reshape them.

- **Q4.2** To what corresponds the dimension of the beamlet matrix?

>*Answer here*

- **Q4.3** Display the same index of the uncorrected beamlets on the 50% phase CT scan. What do you observe?

> *Answer here*

In [ ]:
# code here

- **Q4.4** What do you need to correct in the uncorrected pre-computed beamlets?

**Hint**: The treatment delivery technique used is pencil beam scanning. \
**Hint 2**: Display the beamlet on the CT while contouring the CTV.

> *Answer here*

## 2. Treatment plan optimization

We will now get into the principal part of the lab: the treatment plan optimization. To guide you through this we will go step by step into the optimization process.

### 2.1 Formulating the optimization problem

In intensity modulated proton therapy (IMPT) we can formulate the optimization problem as follows:

\begin{equation}
    \begin{split}
        \min_{\textbf{x}} \{\sum_j w_j f_j(\mathcal{D})\}\\
        \textrm{s.t.} & \quad \mathcal{D} = \mathcal{B}\textbf{x}\\
        &\quad  x\geq0 \quad \forall x \in \textbf{x}\\
    \end{split}
\end{equation}

where the variables are :
<center>

| Variable | Description | Dimension |
|----------------|----------------|----------------|
| $\textbf{D}$   | Dose distribution in 2D | $\mathbb{R}^{n_x \times n_y}$ |
| $\mathcal{D}$  | Dose distribution flattened | $\mathbb{R}^{N}$ |
| $\mathcal{B}$  | Beamlet matrix | $\mathbb{R}^{N \times m}$ |
| $\textbf{x}$   | Beamlet weights | $\mathbb{R}^m$ |
| $f_j$            | Objective function | $\mathbb{R}$ |
| $\sum_j w_j f_j$ | The total objective function| $\mathbb{R}$ |
| $w_j$          | The weight of each objectif | $\mathbb{R}$ |

</center>

To impose positive weights, we will use the following transformation:
\begin{equation}
x_i := u_i^2
\end{equation}

where $u$ is the new optimization variable.

In this session, we will use three types of objective functions :

\begin{equation}
f_{max}(\mathcal{D}) = \frac{1}{N_r} \sum_i^{N_r} \max\{ 0 ; (d_i - d_{presc}) \}^2
\end{equation}
\begin{equation}
f_{min}(\mathcal{D}) = \frac{1}{N_r} \sum_i^{N_r} \min\{ 0 ; (d_i - d_{presc}) \}^2
\end{equation}
\begin{equation}
f_{max-mean}(\mathcal{D}) = \max\{ 0 ; (\frac{1}{N_r} (\sum_i^{N_r} d_i) - d_{presc}) \}^2
\end{equation}

where $d_i$ is the dose at voxel $i$, $d_{presc}$ is the prescription dose and $N_r$ is the number of voxels in the considered region.

As we will use a gradient descent approach to optimize the treatment plan, we will also need to compute the gradient of the objective function with respect to the optimization variable.
- **Q5.1** Write the gradient of the three above-described cost functions.

**Hint**: You will need to use the chain rule. \
**Hint 2**: Think carefully about the dimensions of the gradient.

You can write Latex equations in Markdown cells the following way:

```markdown
\begin{equation}
(a+b)^2 = a^2 + 2ab + b^2
\end{equation}
```

>*Answer here*

### 2.2 Implementation of the cost function

The constraints you will need to implement are the following:

<center>

| Region | Constraint             |
|----------------|------------------------|
| CTV | $D_{max} < 60 Gy$      |
| CTV | $D_{min} > 50.4 Gy$    |
| Heart | $D_{max-mean} < 26 Gy$ |
| Lungs | $D_{max-mean} < 7 Gy$  |
| Liver | $D_{max-mean} < 32 Gy$ |
| Spinal canal | $D_{max} < 20 Gy$      |
| Body | $D_{max} < 10 Gy$      |

</center>

Those constraints are based on the *QUANTEC* tables that you can find in the following paper: "The Use of Normal Tissue Complication Probability (NTCP) Models in the Clinic" by Marks et al. 2010.
(doi:10.1016/j.ijrobp.2009.07.1754.)

- **Q6.1** Implement the evaluation of the cost function
- **Q6.2** Implement the gradient of the cost function

In [ ]:
# code here

### 2.3 Gradient descent

Now that you have every ingredient to implement the optimization, you can implement the gradient descent algorithm.
- **Q7.1** Imlement the gradient descent algorithm
- **Q7.2** Optimize your plan with $w_j=1 \enspace \forall j$.
- **Q7.3** Vary the step size, what do you observe?
>*Answer here*

**Hint**: The optimization process can be quite slow. Try to stay in matrix form as much as possible.

In [ ]:
# code here

### 2.4 Plan evaluation

Now that you have optimized the treatment plan, you can evaluate it. Therefore, you will use the most common tool of treatment plan evaluation: the Dose Volume Histogram (DVH).

Mathematically, we can describe DVH the following way;

Let :
\begin{equation}
     \kappa : \mathbb{R}^+_0 \rightarrow \mathbb{N}_0 \enspace | \enspace D \rightarrow \kappa(D) \enspace,
\end{equation}

and the bins

\begin{equation}
    \begin{split}
        k \in \mathcal{K} := \{0,...,K\}\\
        \textrm{s.t.} \quad & \kappa(0)=0\\
        & \kappa(D_{max}) = K \enspace. \\
    \end{split}
\end{equation}

Then the DVH for a volume of $N_r$ elementary isotropic voxels in a region r, each receiving a dose $d_i$ is defined by :

\begin{equation}
    h_k = \sum_{i=1}^N \delta_{k, \kappa(d_i)} \enspace,
\end{equation}

\begin{equation}
    \textrm{DVH}_k = \frac{100\%}{N} \sum_{j=k}^K h_j \enspace.
\end{equation}


In words, the DVH is: on the x-axis a certain dose value and on the y-axis the percentage of voxels that received at least this dose.

- **Q8.1** Implement the DVH function \
**Hint**: The functions [np.histogram](https://numpy.org/doc/stable/reference/generated/numpy.histogram.html), [np.cumsum](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html) and [np.flip](https://numpy.org/doc/stable/reference/generated/numpy.flip.html) can be useful.
- **Q8.2** Compute the DVH for the CTV, Heart, Lungs, Liver, and Spinal canal
- **Q8.3** What do you observe with respect to the above-described constraints?
>*Answer here*

In [ ]:
# code here

- **Q8.4** Compute the D98 and D2 for the CTV, Heart, Lungs, Liver, and Spinal canal
- **Q8.5** Compute D98-D2 for the CTV, Heart, Lungs, Liver, and Spinal canal
- **Q8.6** What do you observe with respect to the above-described constraints?
>*Answer here*

In [ ]:
# code here

- **Q8.7** Play with the $w_j$ values to reach the optimal plan.
- **Q8.8** What do you observe with respect to the above-described constraints?
>*Answer here*

In [ ]:
# code here

### 2.5 Phase analysis

Now that you have optimized the treatment plan for the 0% phase, you will explore what happens when you use the 50% phase.

- **Q9.1** Keep the same weights and compute the dose distribution for the 50% phase with the correct beamlets and CT.
- **Q9.2** What do you observe? (Do not forget to display the RTSTRUCTS masks)
>*Answer here*

In [ ]:
# code here

- **Q9.3** Compute the DVH for the 50% phase.
- **Q9.4** Overlay the DVH of the 0% phase and the 50% phase on the same plot.
- **Q9.5** What do you observe?
>*Answer here*

In [ ]:
# code here

Let us now suppose that you have the corrected beamlets for the 50% phase at your disposition but that you do not know in which phase is the patient during the delivery. We will explore some strategies to ensure proper target coverage. Answer the following questions based on the course and some of your own research (do not forget to cite your sources).

- **Q9.6** What strategy can you put in place regarding the treatment delivery to ensure proper target coverage?
>*Answer here*
- **Q9.7** What strategy can you put in place regarding the treatment optimization to ensure proper target coverage?
>*Answer here*
- **Q9.8** Write a new cost function that would ensure proper target coverage.
>*Answer here*

## A. Bonuses

In this section, we will propose you some bonus questions. You can choose to answer to one of them (or all, but it will not get you more points).

The answers to these questions will be based on your own research, therefore remember to cite your sources. (Do not worry, the following questions will not ask you to much research).

### A.1 Tumor control probability and normal tissue complication probability

The tumor control probability (TCP) is a measure of the probability of tumor control after treatment. As there are numerous way of expressing the TCP, we will ask you to focus on the one based on the *linear quadratic surviving fraction*.

- **QA1.1** Write the two main TCP models that exist based on the linear quadratic surviving fraction and describe their parameters (with their units) as their differences.\
>*Answer here*
- **QA1.2** Implement the two models and plot the TCP as a function of the dose for the 0% and 50% phases.
- **QA1.3** What do you observe?
>*Answer here*


In [ ]:
# code here

Now you will explore the normal tissue complication probability (NTCP). Again as there are even more expression for the NTCP you will focus on the one based on the linear quadratic surviving fraction.

- **QA1.4** What are the two type of organs at risk that exist and how do they differ?
- **QA1.5** What are their respective NTCP model ?
>*Answer here*
-  **QA1.6** Compute the NTCP for the heart, lungs, liver and spinal canal using the correct model each time


In [ ]:
# code here

### A.2 Equivalent Uniform Dose

In this question, you will explore the concept of equivalent uniform dose (EUD).

- **QA2.1** What is the paradygm of the equivalent uniform dose?
>*Answer here*

You will now compare to type of fomulation of the equivalent uniform dose : the EUD based on the *linear quadratic surviving fraction* and the EUD based on the *generalized equivalent uniform dose* (also called gEUD).
- **QA2.3** What are their respective formula and what do their parameters mean?
>*Answer here*
- **QA2.4** Implement the two models of EUD for the CTV, heart, lungs, liver and spinal canal for the 0% and 50% phases.
- **QA2.5** What do you observe?
>*Answer here*

In [ ]:
# code here

## Instruction for submission

- This notebook is due on **Sunday, May 25, at 23:59:59**.
- Send your filled notebook be email to the two assisants with the naming convention: **Lastname1_Lastname2_LGBIO2070_Lab3.ipynb**.
- To not forget to write your names at the beginning of the notebook.
- All the cells should be pre-run, and the notebook should be clean.
- Do not send the numpy and DICOM files, only the notebook.
- Keep the numpy data under the "data/" folder and the DICOM under the "CT/" folder.
